<p align='center'><img src="img/GausQash.png" width="250"></p>

# GausQash-QKDC (Gaussian Hash Circuit)
- quantum hash generator based on [qash-qkdc](https://github.com/TimeMelt/qash-qkdc)
- this notebook uses the Xanadu-Strawberryfields SDK
- compatible with photonic simulators/devices

In [1]:
import strawberryfields as sf
from strawberryfields import ops
import numpy as np
import qkdc_helper

### Define Hash Circuit Functions
1. set thermal quantum state
2. photonic rotation
3. displacement channel
4. thermal loss channel
5. beamsplitter operation

In [2]:
def setThermalState(input, wires):
    for c,q in enumerate(input):
        n = np.tan(q) * np.cos(q)
        ops.Thermal(n) | wires[c]

def rotationStep(input, wires):
    for c,q in enumerate(input):
        phi = q * np.tan(q)
        ops.Rgate(phi) | wires[c]

def displaceStep(input, wires):
    for c,q in enumerate(input):
        r = q
        phi = np.tan(r) * np.cos(r)
        ops.Dgate(r,phi) | wires[c]

def thermalLoss(input, wires):
    for c,q in enumerate(input):
        T = 0.25
        nbar = np.tan(q)
        ops.ThermalLossChannel(T, nbar) | wires[c]

def beamSplitter(input, wires):
    for c,q in enumerate(input):
        theta = q * (input.size * np.tan(q))
        phi = q * (input.size * np.tan(q))
        if c == input.size or c == input.size-1:
            pass
        else:
            ops.BSgate(theta,phi) | [wires[c], wires[c+1]]

### Prepare Data to Hash
- convert string defined as 'text' to numerical values
- pad based on value of 'pad_length'
- define number of wires based on total length of input 
    - len(text) + pad_length
    - equal to length of string input plus padding length
- print input after numerical conversion
- print number of wires 

In [3]:
text = 'test'
pad_length = 0
input = qkdc_helper.createAndPad(text,pad_length)
num_wires = len(input)

print(f"converted input: {input}")
print(f"number of wires: {num_wires}")

converted input: [1.16 1.01 1.15 1.16]
number of wires: 4


### Create Quantum Hashing Circuit

In [4]:
program = sf.Program(num_wires)

def runProg(engine, program):
    with program.context as wires:
        setThermalState(input, wires)
        rotationStep(input, wires)
        displaceStep(input, wires)
        thermalLoss(input, wires)
        beamSplitter(input, wires)
        ops.MeasureThreshold() | wires
    
    return engine.run(program, shots=1)

### Define Quantum Simulation Engine
- only compatible with gaussian backend 
    - thermal loss channel does not support fock backends

In [5]:
engine = sf.Engine("gaussian", backend_options={})

### Run Circuit

In [6]:
result = runProg(engine, program)

### Show Output
- calculate number expectation for each wire
- convert number expectations to hex and/or base64 format

In [7]:
out_list = np.array([])
for i in range(num_wires):
    out_list = np.append(out_list, result.state.number_expectation(modes=[i]))

print(f"hex output: {qkdc_helper.processOutput(out_list,'hex')}")
print(f"base64 output: {qkdc_helper.processOutput(out_list,'base64')}")

hex output: 6eccadfd0dcb40b503dfbc268a364081bd8bfec5882240ad8c228e71cf58408200790193bcbb40ae399ac550bf44406b6aa854955c844085d1c565176692
base64 output: NmVjY2FkZmQwZGNiNDBiNTAzZGZiYzI2OGEzNjQwODFiZDhiZmVjNTg4MjI0MGFkOGMyMjhlNzFjZjU4NDA4MjAwNzkwMTkzYmNiYjQwYWUzOTlhYzU1MGJmNDQ0MDZiNmFhODU0OTU1Yzg0NDA4NWQxYzU2NTE3NjY5Mg==

